<a href="https://colab.research.google.com/github/garcia-nacho/DNA-RNN/blob/main/DNA_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installation of Libraries
!pip install zstandard
!pip install datasets
!pip install transformers
!pip install huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from huggingface_hub import notebook_login
from transformers import PushToHubCallback


Check Vlasta/DNA_Sentencepiece_vocab_10000_max_tokenlen_45 for tokenizer

In [5]:
#data_files = {"train": "Train/*.tsv.zst", "test": "Test/*.tsv.zst", "validation":"Validation/*.tsv.zst"}
data_files = {"train": "Test/*.tsv.zst", "test": "Validation/*.tsv.zst"}

dna_dataset = load_dataset("garcianacho/human_genome_csv", data_files=data_files)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
notebook_login()

In [6]:
# Generation of Training, Testing and Validation Sets

masking_position=256
input_size=512

x_train=[]
x_test=[]

y_train=[]
y_test=[]

def dna_tokenizer(x,masking_position):
  x = list(map(lambda x: x.replace('A', '0'), x))
  x = list(map(lambda x: x.replace('T', '1'), x))
  x = list(map(lambda x: x.replace('C', '2'), x))
  x = list(map(lambda x: x.replace('G', '3'), x))
  x = list(map(int, x))
  #Masking
  if masking_position > -1:
    x[masking_position]=0


  return(x)

for i in dna_dataset['train']['seq']:
  x_train.append(dna_tokenizer(i,masking_position))

for i in dna_dataset['train']['Y']:
  y_train.append(dna_tokenizer(i,-1))


for i in dna_dataset['test']['seq']:
  x_test.append(dna_tokenizer(i,masking_position))

for i in dna_dataset['test']['Y']:
  y_test.append(dna_tokenizer(i,-1))

x_train=np.array(x_train)
x_test=np.array(x_test)
x_train=x_train[:,:input_size]
x_test=x_test[:,:input_size]

y_train_ohe=tf.keras.utils.to_categorical(y_train, 4)
y_test_ohe=tf.keras.utils.to_categorical(y_test, 4)

#Uncomment when validation is included
#x_val=[]
#y_val=[]

#for i in dna_dataset['validation']['seq']:
#  x_val.append(dna_tokenizer(i,masking_position))

#for i in dna_dataset['validation']['Y']:
#  y_val.append(dna_tokenizer(i,-1))

#x_val=np.array(x_val)

In [ ]:
#Model option 1

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(5, 64, input_length=input_size, mask_zero=False))
model.add(tf.keras.layers.Conv1D(64, kernel_size=4, activation='relu', padding='same', strides=1))
model.add(tf.keras.layers.Conv1D(96, kernel_size=8, activation='relu', padding='same', strides=1))
model.add(tf.keras.layers.Conv1D(128, kernel_size=12, activation='relu', padding='same', strides=1))

model.add(tf.keras.layers.Conv1D(64, kernel_size=4, activation='relu', padding='same', strides=1))
model.add(tf.keras.layers.Conv1D(96, kernel_size=8, activation='relu', padding='same', strides=1))
model.add(tf.keras.layers.Conv1D(128, kernel_size=12, activation='relu', padding='same', strides=1))

model.add(tf.keras.layers.Conv1D(64, kernel_size=4, activation='relu', padding='same', strides=1))
model.add(tf.keras.layers.Conv1D(96, kernel_size=8, activation='relu', padding='same', strides=1))
model.add(tf.keras.layers.Conv1D(128, kernel_size=12, activation='relu', padding='same', strides=1))

model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.Dense(28))
model.add(tf.keras.layers.Dense(4, activation='softmax'))


model.summary()




Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 51, 64)            320       
                                                                 
 conv1d_81 (Conv1D)          (None, 51, 64)            16448     
                                                                 
 conv1d_82 (Conv1D)          (None, 51, 96)            49248     
                                                                 
 conv1d_83 (Conv1D)          (None, 51, 128)           147584    
                                                                 
 conv1d_84 (Conv1D)          (None, 51, 64)            32832     
                                                                 
 conv1d_85 (Conv1D)          (None, 51, 96)            49248     
                                                                 
 conv1d_86 (Conv1D)          (None, 51, 128)          

In [ ]:
batch_size = 128
epochs = 100

model.compile(loss="categorical_crossentropy", optimizer="adagrad", metrics=["categorical_accuracy"])

model.fit(x_train, y_train_ohe, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test_ohe))


In [7]:
#Transfomer
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = tf.keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions



embed_dim = 32  # Embedding size for each token
num_heads = 20  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = tf.keras.layers.Input(shape=(input_size,))
embedding_layer = TokenAndPositionEmbedding(input_size, 4, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(20, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(4, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512)]             0         
                                                                 
 token_and_position_embeddi  (None, 512, 32)           16512     
 ng (TokenAndPositionEmbedd                                      
 ing)                                                            
                                                                 
 transformer_block (Transfo  (None, 512, 32)           86112     
 rmerBlock)                                                      
                                                                 
 global_average_pooling1d (  (None, 32)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dropout_2 (Dropout)         (None, 32)                0     

In [ ]:
model.compile(optimizer="adagrad", loss="categorical_crossentropy", metrics=["categorical_accuracy"])
history = model.fit(
    x_train, y_train_ohe, batch_size=128, epochs=50, validation_data=(x_test, y_test_ohe)
)



In [ ]:
model.predict(
    x_test,
    batch_size=None,
    verbose="auto",
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)

50/50 [==============================] - 0s 3ms/step


array([[0.28281558, 0.28355348, 0.2187776 , 0.21485336],
       [0.28283635, 0.2828059 , 0.2188721 , 0.21548566],
       [0.2830744 , 0.28295845, 0.21865971, 0.21530741],
       ...,
       [0.28291354, 0.2833809 , 0.21870762, 0.21499793],
       [0.28295994, 0.2827781 , 0.2186291 , 0.21563284],
       [0.2829645 , 0.28353325, 0.21862313, 0.21487913]], dtype=float32)

In [ ]:
#Model Dilated Convolution
model = tf.keras.Sequential()

model.add(tf.keras.layers.Embedding(5, 512, input_length=input_size, mask_zero=False))

model.add(tf.keras.layers.Conv1D(512, kernel_size=9, activation='relu', padding='same', strides=1, dilation_rate=1))
model.add(tf.keras.layers.LayerNormalization())
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.LayerNormalization())

model.add(tf.keras.layers.Conv1D(512, kernel_size=9, activation='relu', padding='same', strides=1, dilation_rate=2))
model.add(tf.keras.layers.LayerNormalization())
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.LayerNormalization())

model.add(tf.keras.layers.Conv1D(512, kernel_size=9, activation='relu', padding='same', strides=1, dilation_rate=4))
model.add(tf.keras.layers.LayerNormalization())
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.LayerNormalization())

model.summary()


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 512, 512)          2560      
                                                                 
 conv1d_13 (Conv1D)          (None, 512, 512)          2359808   
                                                                 
 layer_normalization_14 (La  (None, 512, 512)          1024      
 yerNormalization)                                               
                                                                 
 dense_11 (Dense)            (None, 512, 512)          262656    
                                                                 
 layer_normalization_15 (La  (None, 512, 512)          1024      
 yerNormalization)                                               
                                                                 
 conv1d_14 (Conv1D)          (None, 512, 512)        

In [19]:
input_size=512
embed_dim = 512  # Embedding size for each token
d_rate = 20  # Number of attention heads
ff_dim = 128  # Hidden layer size in feed forward network inside transformer


class CNN1D_Block(tf.keras.layers.Layer):
    def __init__(self, embed_dim, d_rate, ff_dim, rate=0.1):
        super().__init__()
        self.cnn = tf.keras.layers.Conv1D(512, kernel_size=9, activation='relu', padding='same', strides=1, dilation_rate=d_rate)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.cnn(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


# Model
inputs = tf.keras.layers.Input(shape=(input_size,))

embedding_layer=tf.keras.layers.Embedding(5, 512, input_length=input_size, mask_zero=False)
cnn_block = CNN1D_Block(embed_dim, d_rate=2, ff_dim=ff_dim)

x = embedding_layer(inputs)

cnn_block = CNN1D_Block(embed_dim, d_rate=1, ff_dim=ff_dim)
x = cnn_block(x)

cnn_block = CNN1D_Block(embed_dim, d_rate=2, ff_dim=ff_dim)
x = cnn_block(x)

cnn_block = CNN1D_Block(embed_dim, d_rate=4, ff_dim=ff_dim)
x = cnn_block(x)

cnn_block = CNN1D_Block(embed_dim, d_rate=8, ff_dim=ff_dim)
x = cnn_block(x)

cnn_block = CNN1D_Block(embed_dim, d_rate=16, ff_dim=ff_dim)
x = cnn_block(x)

x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(20, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(4, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 512)]             0         
                                                                 
 embedding_12 (Embedding)    (None, 512, 512)          2560      
                                                                 
 cnn1d__block_19 (CNN1D_Blo  (None, 512, 512)          2493568   
 ck)                                                             
                                                                 
 cnn1d__block_20 (CNN1D_Blo  (None, 512, 512)          2493568   
 ck)                                                             
                                                                 
 cnn1d__block_21 (CNN1D_Blo  (None, 512, 512)          2493568   
 ck)                                                             
                                                           